Comece importando as bibliotecas necessárias:
- pyspark.sql
- random
- faker
- pymysql para lidar com a conexão ao banco de dados MariaDB.

In [14]:
import pymysql
import random

from faker import Faker
from pyspark.sql import SparkSession

Inicialize uma sessão Spark, que é a maneira de interagir com os dados no Spark

In [16]:
spark = SparkSession.builder.master(
    "local[1]"
    ).appName(
        'test'
    ).config(
        "spark.jars", "mariadb-java-client-1.8.0.jar"
    ).getOrCreate()
spark

In [7]:


fake = Faker()
num_rows = 1000
data = [
    {
        "id": _ ,
        "firstname": fake.first_name(),
        "middlename": fake.last_name(),
        "lastname": fake.last_name(),
        "gender": random.choice(["Male", "Female"]),
        "salary": round(random.uniform(1000.00, 10000.00), 2)
    } for _ in range(num_rows)
]

df = spark.createDataFrame(data)

df.show()

23/08/19 01:43:04 INFO CodeGenerator: Code generated in 296.226812 ms
23/08/19 01:43:04 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/08/19 01:43:04 INFO DAGScheduler: Got job 0 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/08/19 01:43:04 INFO DAGScheduler: Final stage: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0)
23/08/19 01:43:04 INFO DAGScheduler: Parents of final stage: List()
23/08/19 01:43:04 INFO DAGScheduler: Missing parents: List()
23/08/19 01:43:04 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[6] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
23/08/19 01:43:05 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 13.7 KiB, free 434.4 MiB)
23/08/19 01:43:05 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 6.9 KiB, free 434.4 MiB)
23/08/19 01:43:05 INFO BlockManagerInfo: Added broadcast_0

+---------+------+---+---------+----------+-------+
|firstname|gender| id| lastname|middlename| salary|
+---------+------+---+---------+----------+-------+
| Savannah|Female|  0|     Reid|     Marks|5658.63|
|   Latoya|Female|  1|   Rogers|     Hines|4577.64|
|     John|  Male|  2|Robertson|      Wood|7829.49|
|  Richard|Female|  3|    Chase|      Meza|5397.59|
| Veronica|Female|  4|  Hubbard| Middleton|3384.88|
| Courtney|Female|  5|   Greene|     Parks|3496.85|
|    David|Female|  6|  Jackson|      Reed|9224.09|
|     Jeff|  Male|  7|   Taylor|  Matthews|7838.03|
|     Lori|Female|  8|  Sanchez|     Stone|3960.79|
|   Nicole|  Male|  9|   Garcia|Strickland|5457.62|
|   Connie|Female| 10|    Davis|     Moore|3276.73|
|    Jason|  Male| 11|      Lee|      Shah|7478.96|
|    Karen|Female| 12| Schwartz| Hernandez|6929.37|
|    Aaron|Female| 13|    Klein|      Cruz|3074.48|
|  Vanessa|Female| 14|   Wilson|Cunningham|7721.76|
| Kathleen|  Male| 15|    Smith|     Mayer|5089.07|
|     Dana| 

23/08/19 01:43:05 INFO Executor: Finished task 0.0 in stage 0.0 (TID 0). 3120 bytes result sent to driver
23/08/19 01:43:05 INFO TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 814 ms on 10.0.0.30 (executor driver) (1/1)
23/08/19 01:43:06 INFO TaskSchedulerImpl: Removed TaskSet 0.0, whose tasks have all completed, from pool 
23/08/19 01:43:06 INFO PythonAccumulatorV2: Connected to AccumulatorServer at host: 127.0.0.1 port: 35497
23/08/19 01:43:06 INFO DAGScheduler: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0) finished in 1,099 s
23/08/19 01:43:06 INFO DAGScheduler: Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
23/08/19 01:43:06 INFO TaskSchedulerImpl: Killing all running tasks in stage 0: Stage finished
23/08/19 01:43:06 INFO DAGScheduler: Job 0 finished: showString at NativeMethodAccessorImpl.java:0, took 1,137242 s
23/08/19 01:43:06 INFO CodeGenerator: Code generated in 29.202428 ms            


In [13]:
import time
import pymysql.cursors

# global variable
def process_row(data):
    connection_properties = brConnect.value
    db_conn = pymysql.connect(
        host=connection_properties.get("host"),
        user=connection_properties.get("user"),
        password=connection_properties.get("password"),
        database=connection_properties.get("database"),
        port=3306
    )
    cursor = db_conn.cursor()
    tabela = connection_properties.get("tabela")
    columns = connection_properties.get("colunas")
    placeholders = ", ".join(["%s"] * len(columns))
    columns_str = ", ".join(columns)

    query = f"""
    INSERT INTO {tabela} ({columns_str})
        VALUES ({placeholders})
        ON DUPLICATE KEY UPDATE
    """ + ", ".join([f"{column}=VALUES({column})" for column in columns])
    
    cursor.executemany(query, data)
    db_conn.commit()
    cursor.close()

def process_partition_bacth(partition):    
    itemBank = [ tuple(row) for row in partition ]
    yield process_row(itemBank)

sc = spark.sparkContext

user = 'root'
pwd = 'secret'
columns = ["id","firstname","middlename","lastname","gender","salary"]


df = df.select(columns)
connection_properties = {
    "host": '127.0.0.1',
    "user": user,
    "password": pwd,
    "database": "teste",
    "tabela": "test1",
    "colunas": columns
}

start = time.time()

rowsPerPartition = 10

partitions = (df.count() / rowsPerPartition)
df = df.repartition(numPartitions=int(partitions))
print("qtd_lines " + str(df.count()) + " partitions " + str(df.rdd.getNumPartitions()) + " df")

brConnect = sc.broadcast(connection_properties)
df.foreachPartition(process_partition_bacth)

end = time.time()

print(f'Total de tempo executado: {end-start/60} minutes')

23/08/19 01:45:43 INFO CodeGenerator: Code generated in 13.009134 ms
23/08/19 01:45:43 INFO DAGScheduler: Registering RDD 8 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 0
23/08/19 01:45:43 INFO DAGScheduler: Got map stage job 1 (count at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/08/19 01:45:43 INFO DAGScheduler: Final stage: ShuffleMapStage 1 (count at NativeMethodAccessorImpl.java:0)
23/08/19 01:45:43 INFO DAGScheduler: Parents of final stage: List()
23/08/19 01:45:43 INFO DAGScheduler: Missing parents: List()
23/08/19 01:45:43 INFO DAGScheduler: Submitting ShuffleMapStage 1 (MapPartitionsRDD[8] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
23/08/19 01:45:43 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 15.8 KiB, free 434.4 MiB)
23/08/19 01:45:43 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 8.3 KiB, free 434.4 MiB)
23/08/19 01:45:43 INFO BlockManag

qtd_lines 1000 partitions 100 df


23/08/19 01:45:46 INFO PythonRunner: Times: total = 223, boot = -164, init = 374, finish = 13
23/08/19 01:45:46 INFO Executor: Finished task 0.0 in stage 12.0 (TID 106). 3944 bytes result sent to driver
23/08/19 01:45:46 INFO TaskSetManager: Starting task 1.0 in stage 12.0 (TID 107) (10.0.0.30, executor driver, partition 1, NODE_LOCAL, 7363 bytes) 
23/08/19 01:45:46 INFO Executor: Running task 1.0 in stage 12.0 (TID 107)
23/08/19 01:45:46 INFO TaskSetManager: Finished task 0.0 in stage 12.0 (TID 106) in 261 ms on 10.0.0.30 (executor driver) (1/100)
23/08/19 01:45:46 INFO ShuffleBlockFetcherIterator: Getting 1 (717.0 B) non-empty blocks including 1 (717.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) push-merged-local and 0 (0.0 B) remote blocks
23/08/19 01:45:46 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 1 ms
23/08/19 01:45:46 INFO PythonRunner: Times: total = 149, boot = 5, init = 135, finish = 9
23/08/19 01:45:46 INFO Executor: Finished task 1.0 in stage 12.0 (TI

Total de tempo executado: 1664213352.0782793 minutes


23/08/19 01:45:57 INFO PythonRunner: Times: total = 89, boot = 6, init = 79, finish = 4
23/08/19 01:45:57 INFO Executor: Finished task 97.0 in stage 12.0 (TID 203). 3944 bytes result sent to driver
23/08/19 01:45:57 INFO TaskSetManager: Starting task 98.0 in stage 12.0 (TID 204) (10.0.0.30, executor driver, partition 98, NODE_LOCAL, 7363 bytes) 
23/08/19 01:45:57 INFO TaskSetManager: Finished task 97.0 in stage 12.0 (TID 203) in 95 ms on 10.0.0.30 (executor driver) (98/100)
23/08/19 01:45:57 INFO Executor: Running task 98.0 in stage 12.0 (TID 204)
23/08/19 01:45:57 INFO ShuffleBlockFetcherIterator: Getting 1 (593.0 B) non-empty blocks including 1 (593.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) push-merged-local and 0 (0.0 B) remote blocks
23/08/19 01:45:57 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
23/08/19 01:45:57 INFO PythonRunner: Times: total = 84, boot = 5, init = 76, finish = 3
23/08/19 01:45:57 INFO Executor: Finished task 98.0 in stage 12.0 (TID 